# Missing Stats and Visualizations

Using this notebook can help obtain the figures as well as missing statistics presented in the document, using the grids generated by the other notebooks.

First, we import necessary libraries.

In [ ]:
import sys
import os
import netCDF4 as ntf
from pyhdf.SD import SD, SDC
import numpy as np
import math
from netCDF4 import Dataset 
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import cartopy.crs as ccrs
import cartopy
import xesmf as xmf
import xarray as xr
import glob

First, read in your desired grid by providing its filepath, as well as the relavent variable in the NetCDF file.

In [ ]:
fp = '/data0/rm3873/aod_30_T2M.nc'
relavent_var = 'T2M'
cur_regrid = xr.open_dataset(fp)
cur_regrid = cur_regrid.to_array().to_numpy().reshape(365,121,96)

Get the % missing values in current grid (Table 1 in paper)

In [ ]:
np.count_nonzero(np.isnan(cur_regrid))/(365*121*96)*100 

Read in the regridded AOD to get its water mask

In [ ]:
temp_aod = xr.open_dataset("/data0/rm3873/regridded.nc")
temp_aod = temp_aod['aot'].to_numpy()
WATER=-4999

For each non-aquatic grid cell, get the percent of days of the year that are present values

In [ ]:
cur_missing_days = np.full((121,96),np.nan)
for i in range(121):
    for j in range(96):
        if(temp_aod[0][i][j] == WATER):
            continue
            
        cur_day = cur_regrid[:,i,j]
        cur_pres_rat = 100 -len(np.argwhere(np.isnan(cur_day)))/365*100
        cur_missing_days[i][j] = cur_pres_rat
        

Give the figure title name and output file name.

In [ ]:
title = 'AOD Custom Days Present threshold=30% (%)'
fname = 'aod_custom_missing_30_t2m.png'

Sets up figure generation, passes the information into figure, generates and saves it. Used to generate Figures 2 - 10.

In [ ]:
lon=np.arange(68,98,.3125)
lat=np.arange(6,36.25,.25)
cbar_label = "Days Present (%)"
plt.figure(figsize=(16,16),facecolor='white', dpi=80)
ax = plt.axes(projection=ccrs.Mercator(min_latitude=5,max_latitude=40,central_longitude=84))
ax.set_extent([60,100,5,40])
ax.add_feature(cartopy.feature.LAND)
ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=2, color='gray', alpha=0.5, linestyle='--')
ax.add_feature(cartopy.feature.OCEAN)
ax.add_feature(cartopy.feature.COASTLINE,linewidth=0.3)
ax.add_feature(cartopy.feature.BORDERS, linestyle=':',linewidth=0.3)
levels = np.linspace(0, 100, 11)

plt.contourf(lon,lat,cur_missing_days,levels=levels, cmap='viridis',transform=ccrs.PlateCarree(),extend='min')
plt.colorbar(label=cbar_label,shrink=0.7)

plt.title(title)
plt.savefig(fname)
plt.show()